# TV Script Generation

In this project, you'll generate your own [Seinfeld](https://en.wikipedia.org/wiki/Seinfeld) TV scripts using RNNs.  You'll be using part of the [Seinfeld dataset](https://www.kaggle.com/thec03u5/seinfeld-chronicles#scripts.csv) of scripts from 9 seasons.  The Neural Network you'll build will generate a new ,"fake" TV script, based on patterns it recognizes in this training data.

## Get the Data

The data is already provided for you in `./data/Seinfeld_Scripts.txt` and you're encouraged to open that file and look at the text. 
>* As a first step, we'll load in this data and look at some samples. 
* Then, you'll be tasked with defining and training an RNN to generate a new script!

In [3]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# load in data
import helper
data_dir = './data/Seinfeld_Scripts.txt'
text = helper.load_data(data_dir)

## Explore the Data
Play around with `view_line_range` to view different parts of the data. This will give you a sense of the data you'll be working with. You can see, for example, that it is all lowercase text, and each new line of dialogue is separated by a newline character `\n`.

In [4]:
view_line_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))

lines = text.split('\n')
print('Number of lines: {}'.format(len(lines)))
word_count_line = [len(line.split()) for line in lines]
print('Average number of words in each line: {}'.format(np.average(word_count_line)))

print()
print('The lines {} to {}:'.format(*view_line_range))
print('\n'.join(text.split('\n')[view_line_range[0]:view_line_range[1]]))

Dataset Stats
Roughly the number of unique words: 46367
Number of lines: 109233
Average number of words in each line: 5.544240293684143

The lines 0 to 10:
jerry: do you know what this is all about? do you know, why were here? to be out, this is out...and out is one of the single most enjoyable experiences of life. people...did you ever hear people talking about we should go out? this is what theyre talking about...this whole thing, were all out now, no one is home. not one person here is home, were all out! there are people trying to find us, they dont know where we are. (on an imaginary phone) did you ring?, i cant find him. where did he go? he didnt tell me where he was going. he must have gone out. you wanna go out you get ready, you pick out the clothes, right? you take the shower, you get all ready, get the cash, get your friends, the car, the spot, the reservation...then youre standing around, what do you do? you go we gotta be getting back. once youre out, you wanna get back! y

---
## Implement Pre-processing Functions
The first thing to do to any dataset is pre-processing.  Implement the following pre-processing functions below:
- Lookup Table
- Tokenize Punctuation

### Lookup Table
To create a word embedding, you first need to transform the words to ids.  In this function, create two dictionaries:
- Dictionary to go from the words to an id, we'll call `vocab_to_int`
- Dictionary to go from the id to word, we'll call `int_to_vocab`

Return these dictionaries in the following **tuple** `(vocab_to_int, int_to_vocab)`

In [5]:
import problem_unittests as tests
from collections import Counter

def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    # TODO: Implement Function
    counts = Counter(text)
    vocab = sorted(counts, key=counts.get, reverse=True)
    
    vocab_to_int = {word: i for i, word in enumerate(vocab)}
    int_to_vocab = {i: word for word, i in vocab_to_int.items()}
    # return tuple
    return (vocab_to_int, int_to_vocab)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_create_lookup_tables(create_lookup_tables)

Tests Passed


### Tokenize Punctuation
We'll be splitting the script into a word array using spaces as delimiters.  However, punctuations like periods and exclamation marks can create multiple ids for the same word. For example, "bye" and "bye!" would generate two different word ids.

Implement the function `token_lookup` to return a dict that will be used to tokenize symbols like "!" into "||Exclamation_Mark||".  Create a dictionary for the following symbols where the symbol is the key and value is the token:
- Period ( **.** )
- Comma ( **,** )
- Quotation Mark ( **"** )
- Semicolon ( **;** )
- Exclamation mark ( **!** )
- Question mark ( **?** )
- Left Parentheses ( **(** )
- Right Parentheses ( **)** )
- Dash ( **-** )
- Return ( **\n** )

This dictionary will be used to tokenize the symbols and add the delimiter (space) around it.  This separates each symbols as its own word, making it easier for the neural network to predict the next word. Make sure you don't use a value that could be confused as a word; for example, instead of using the value "dash", try using something like "||dash||".

In [6]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenized dictionary where the key is the punctuation and the value is the token
    """
    # TODO: Implement Function
    punctuation_dict = {
        '.': '||period||',
        ',': '||comma||',
        '"': '||quotation_mark||',
        ';': '||semicolon||',
        '!': '||exclamation_mark||',
        '?': '||question_mark||',
        '(': '||left_parentheses||',
        ')': '||right_parentheses||',
        '-': '||dash||',
        '\n': '||return||'
    }
        
    return punctuation_dict

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_tokenize(token_lookup)

Tests Passed


## Pre-process all the data and save it

Running the code cell below will pre-process all the data and save it to file. You're encouraged to lok at the code for `preprocess_and_save_data` in the `helpers.py` file to see what it's doing in detail, but you do not need to change this code.

In [7]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# pre-process training data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import problem_unittests as tests

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

## Build the Neural Network
In this section, you'll build the components necessary to build an RNN by implementing the RNN Module and forward and backpropagation functions.

### Check Access to GPU

In [2]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import torch

# Check for a GPU
train_on_gpu = torch.cuda.is_available()
if not train_on_gpu:
    print('No GPU found. Please use a GPU to train your neural network.')

No GPU found. Please use a GPU to train your neural network.


## Input
Let's start with the preprocessed input data. We'll use [TensorDataset](http://pytorch.org/docs/master/data.html#torch.utils.data.TensorDataset) to provide a known format to our dataset; in combination with [DataLoader](http://pytorch.org/docs/master/data.html#torch.utils.data.DataLoader), it will handle batching, shuffling, and other dataset iteration functions.

You can create data with TensorDataset by passing in feature and target tensors. Then create a DataLoader as usual.
```
data = TensorDataset(feature_tensors, target_tensors)
data_loader = torch.utils.data.DataLoader(data, 
                                          batch_size=batch_size)
```

### Batching
Implement the `batch_data` function to batch `words` data into chunks of size `batch_size` using the `TensorDataset` and `DataLoader` classes.

>You can batch words using the DataLoader, but it will be up to you to create `feature_tensors` and `target_tensors` of the correct size and content for a given `sequence_length`.

For example, say we have these as input:
```
words = [1, 2, 3, 4, 5, 6, 7]
sequence_length = 4
```

Your first `feature_tensor` should contain the values:
```
[1, 2, 3, 4]
```
And the corresponding `target_tensor` should just be the next "word"/tokenized word value:
```
5
```
This should continue with the second `feature_tensor`, `target_tensor` being:
```
[2, 3, 4, 5]  # features
6             # target
```

In [9]:
import numpy as np
seq_length = 200

In [10]:
targets = np.array(int_text[seq_length:], dtype=int)

In [15]:
targets

array([51, 49,  2, ...,  1,  0,  0])

In [16]:
targets1 = np.array([int_text[i+seq_length] for i, f in enumerate(int_text[:-seq_length])])

In [17]:
targets1

array([51, 49,  2, ...,  1,  0,  0])

In [13]:
features = np.array([int_text[i:i+seq_length] for i, element in enumerate(int_text[:-seq_length])])

In [14]:
features.shape

(891910, 200)

In [19]:
from torch.utils.data import TensorDataset, DataLoader
import numpy as np


def batch_data(words, sequence_length, batch_size):
    """
    Batch the neural network data using DataLoader
    :param words: The word ids of the TV scripts
    :param sequence_length: The sequence length of each batch
    :param batch_size: The size of each batch; the number of sequences in a batch
    :return: DataLoader with batched data
    """
    # TODO: Implement function
    features = np.array([words[i:i+sequence_length] for i, element in enumerate(words[:-sequence_length])], dtype=int)
    targets = np.array(words[sequence_length:],dtype=int)
    
    # return a dataloader
    data = TensorDataset(torch.from_numpy(features), torch.from_numpy(targets))
    data_loader = DataLoader(data, shuffle=True, batch_size=batch_size)
    return data_loader

# there is no test for this function, but you are encouraged to create
# print statements and tests of your own


### Test your dataloader 

You'll have to modify this code to test a batching function, but it should look fairly similar.

Below, we're generating some test text data and defining a dataloader using the function you defined, above. Then, we are getting some sample batch of inputs `sample_x` and targets `sample_y` from our dataloader.

Your code should return something like the following (likely in a different order, if you shuffled your data):

```
torch.Size([10, 5])
tensor([[ 28,  29,  30,  31,  32],
        [ 21,  22,  23,  24,  25],
        [ 17,  18,  19,  20,  21],
        [ 34,  35,  36,  37,  38],
        [ 11,  12,  13,  14,  15],
        [ 23,  24,  25,  26,  27],
        [  6,   7,   8,   9,  10],
        [ 38,  39,  40,  41,  42],
        [ 25,  26,  27,  28,  29],
        [  7,   8,   9,  10,  11]])

torch.Size([10])
tensor([ 33,  26,  22,  39,  16,  28,  11,  43,  30,  12])
```

### Sizes
Your sample_x should be of size `(batch_size, sequence_length)` or (10, 5) in this case and sample_y should just have one dimension: batch_size (10). 

### Values

You should also notice that the targets, sample_y, are the *next* value in the ordered test_text data. So, for an input sequence `[ 28,  29,  30,  31,  32]` that ends with the value `32`, the corresponding output should be `33`.

In [20]:
# test dataloader

test_text = range(50)
t_loader = batch_data(test_text, sequence_length=5, batch_size=10)

data_iter = iter(t_loader)
sample_x, sample_y = data_iter.next()

print(sample_x.shape)
print(sample_x)
print()
print(sample_y.shape)
print(sample_y)

torch.Size([10, 5])
tensor([[28, 29, 30, 31, 32],
        [22, 23, 24, 25, 26],
        [ 5,  6,  7,  8,  9],
        [ 7,  8,  9, 10, 11],
        [41, 42, 43, 44, 45],
        [ 0,  1,  2,  3,  4],
        [ 2,  3,  4,  5,  6],
        [18, 19, 20, 21, 22],
        [ 9, 10, 11, 12, 13],
        [36, 37, 38, 39, 40]])

torch.Size([10])
tensor([33, 27, 10, 12, 46,  5,  7, 23, 14, 41])


---
## Build the Neural Network
Implement an RNN using PyTorch's [Module class](http://pytorch.org/docs/master/nn.html#torch.nn.Module). You may choose to use a GRU or an LSTM. To complete the RNN, you'll have to implement the following functions for the class:
 - `__init__` - The initialize function. 
 - `init_hidden` - The initialization function for an LSTM/GRU hidden state
 - `forward` - Forward propagation function.
 
The initialize function should create the layers of the neural network and save them to the class. The forward propagation function will use these layers to run forward propagation and generate an output and a hidden state.

**The output of this model should be the *last* batch of word scores** after a complete sequence has been processed. That is, for each input sequence of words, we only want to output the word scores for a single, most likely, next word.

### Hints

1. Make sure to stack the outputs of the lstm to pass to your fully-connected layer, you can do this with `lstm_output = lstm_output.contiguous().view(-1, self.hidden_dim)`
2. You can get the last batch of word scores by shaping the output of the final, fully-connected layer like so:

```
# reshape into (batch_size, seq_length, output_size)
output = output.view(batch_size, -1, self.output_size)
# get last batch
out = output[:, -1]
```

In [23]:
import torch.nn as nn

class RNN(nn.Module):
    
    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5):
        """
        Initialize the PyTorch RNN Module
        :param vocab_size: The number of input dimensions of the neural network (the size of the vocabulary)
        :param output_size: The number of output dimensions of the neural network
        :param embedding_dim: The size of embeddings, should you choose to use them        
        :param hidden_dim: The size of the hidden layer outputs
        :param dropout: dropout to add in between LSTM/GRU layers
        """
        super(RNN, self).__init__()
        # TODO: Implement function
        
        # set class variables
        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        
        # define model layers
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, dropout=dropout, batch_first=True)
        
        self.dropout = nn.Dropout(0.3)
        self.fc = nn.Linear(hidden_dim, output_size)
        
    
    
    def forward(self, nn_input, hidden):
        """
        Forward propagation of the neural network
        :param nn_input: The input to the neural network
        :param hidden: The hidden state        
        :return: Two Tensors, the output of the neural network and the latest hidden state
        """
        # TODO: Implement function
        batch_size = nn_input.size(0)
        x = nn_input.long()
        embed = self.embedding(x)
        lstm_out, hidden = self.lstm(embed, hidden)
        out = self.dropout(lstm_out)
        out = out.contiguous().view(-1, self.hidden_dim)
        out = self.fc(out)
        out = out.view(batch_size, -1, self.output_size)
        out = out[:, -1]
        # return one batch of output word scores and the hidden state
        return out, hidden
    
    
    def init_hidden(self, batch_size):
        '''
        Initialize the hidden state of an LSTM/GRU
        :param batch_size: The batch_size of the hidden state
        :return: hidden state of dims (n_layers, batch_size, hidden_dim)
        '''
        # Implement function
        weight = next(self.parameters()).data
        
        # initialize hidden state with zero weights, and move to GPU if available
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        
        return hidden

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_rnn(RNN, train_on_gpu)

Tests Passed


### Define forward and backpropagation

Use the RNN class you implemented to apply forward and back propagation. This function will be called, iteratively, in the training loop as follows:
```
loss = forward_back_prop(decoder, decoder_optimizer, criterion, inp, target)
```

And it should return the average loss over a batch and the hidden state returned by a call to `RNN(inp, hidden)`. Recall that you can get this loss by computing it, as usual, and calling `loss.item()`.

**If a GPU is available, you should move your data to that GPU device, here.**

In [25]:
def forward_back_prop(rnn, optimizer, criterion, inp, target, hidden):
    """
    Forward and backward propagation on the neural network
    :param rnn: The PyTorch Module that holds the neural network
    :param optimizer: The PyTorch optimizer for the neural network
    :param criterion: The PyTorch loss function
    :param inp: A batch of input to the neural network
    :param target: The target output for the batch of input
    :return: The loss and the latest hidden state Tensor
    """
    
    # TODO: Implement Function
    
    # move data to GPU, if available
    if train_on_gpu:
        inp, target = inp.cuda(), target.cuda()
    
    # perform backpropagation and optimization
    hidden = tuple([each.data for each in hidden])
    rnn.zero_grad()
    output, hidden = rnn(inp, hidden)
    
    loss = criterion(output, target)
    loss.backward()
    optimizer.step()

    # return the loss over a batch and the hidden state produced by our model
    return loss.item(), hidden

# Note that these tests aren't completely extensive.
# they are here to act as general checks on the expected outputs of your functions
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_forward_back_prop(RNN, forward_back_prop, train_on_gpu)

Tests Passed


## Neural Network Training

With the structure of the network complete and data ready to be fed in the neural network, it's time to train it.

### Train Loop

The training loop is implemented for you in the `train_decoder` function. This function will train the network over all the batches for the number of epochs given. The model progress will be shown every number of batches. This number is set with the `show_every_n_batches` parameter. You'll set this parameter along with other parameters in the next section.

In [26]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""

def train_rnn(rnn, batch_size, optimizer, criterion, n_epochs, show_every_n_batches=100):
    batch_losses = []
    
    rnn.train()

    print("Training for %d epoch(s)..." % n_epochs)
    for epoch_i in range(1, n_epochs + 1):
        
        # initialize hidden state
        hidden = rnn.init_hidden(batch_size)
        
        for batch_i, (inputs, labels) in enumerate(train_loader, 1):
            
            # make sure you iterate over completely full batches, only
            n_batches = len(train_loader.dataset)//batch_size
            if(batch_i > n_batches):
                break
            
            # forward, back prop
            loss, hidden = forward_back_prop(rnn, optimizer, criterion, inputs, labels, hidden)          
            # record loss
            batch_losses.append(loss)

            # printing loss stats
            if batch_i % show_every_n_batches == 0:
                print('Epoch: {:>4}/{:<4}  Loss: {}\n'.format(
                    epoch_i, n_epochs, np.average(batch_losses)))
                batch_losses = []

    # returns a trained rnn
    return rnn

### Hyperparameters

Set and train the neural network with the following parameters:
- Set `sequence_length` to the length of a sequence.
- Set `batch_size` to the batch size.
- Set `num_epochs` to the number of epochs to train for.
- Set `learning_rate` to the learning rate for an Adam optimizer.
- Set `vocab_size` to the number of unique tokens in our vocabulary.
- Set `output_size` to the desired size of the output.
- Set `embedding_dim` to the embedding dimension; smaller than the vocab_size.
- Set `hidden_dim` to the hidden dimension of your RNN.
- Set `n_layers` to the number of layers/cells in your RNN.
- Set `show_every_n_batches` to the number of batches at which the neural network should print progress.

If the network isn't getting the desired results, tweak these parameters and/or the layers in the `RNN` class.

In [27]:
# Data params
# Sequence Length
sequence_length =  15  # of words in a sequence
# Batch Size
batch_size = 256

# data loader - do not change
train_loader = batch_data(int_text, sequence_length, batch_size)

In [28]:
# Training parameters
# Number of Epochs
num_epochs = 2
# Learning Rate
learning_rate = 0.001

# Model parameters
# Vocab size
vocab_size = len(vocab_to_int)
# Output size
output_size = len(vocab_to_int)
# Embedding Dimension
embedding_dim =  200
# Hidden Dimension
hidden_dim = 256
# Number of RNN Layers
n_layers = 2

# Show stats for every n number of batches
show_every_n_batches = 2

### Train
In the next cell, you'll train the neural network on the pre-processed data.  If you have a hard time getting a good loss, you may consider changing your hyperparameters. In general, you may get better results with larger hidden and n_layer dimensions, but larger models take a longer time to train. 
> **You should aim for a loss less than 3.5.** 

You should also experiment with different sequence lengths, which determine the size of the long range dependencies that a model can learn.

In [29]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""

# create model and move to gpu if available
rnn = RNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5)
if train_on_gpu:
    rnn.cuda()

# defining loss and optimization functions for training
optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

# training the model
trained_rnn = train_rnn(rnn, batch_size, optimizer, criterion, num_epochs, show_every_n_batches)

# saving the trained model
helper.save_model('./save/trained_rnn', trained_rnn)
print('Model Trained and Saved')

Training for 2 epoch(s)...
Epoch:    1/2     Loss: 9.957801818847656

Epoch:    1/2     Loss: 9.906312942504883

Epoch:    1/2     Loss: 9.79367446899414

Epoch:    1/2     Loss: 9.380387306213379

Epoch:    1/2     Loss: 8.439937591552734

Epoch:    1/2     Loss: 7.809945106506348

Epoch:    1/2     Loss: 7.01547908782959

Epoch:    1/2     Loss: 6.364347457885742

Epoch:    1/2     Loss: 6.106651782989502

Epoch:    1/2     Loss: 6.3118298053741455

Epoch:    1/2     Loss: 6.151082754135132

Epoch:    1/2     Loss: 6.2213294506073

Epoch:    1/2     Loss: 6.126859426498413

Epoch:    1/2     Loss: 6.079297304153442

Epoch:    1/2     Loss: 6.374754190444946

Epoch:    1/2     Loss: 6.135115385055542

Epoch:    1/2     Loss: 6.255238056182861

Epoch:    1/2     Loss: 6.103996992111206

Epoch:    1/2     Loss: 6.479295253753662

Epoch:    1/2     Loss: 6.235327959060669

Epoch:    1/2     Loss: 6.05108642578125

Epoch:    1/2     Loss: 6.247842788696289

Epoch:    1/2     Loss: 6.11193

Epoch:    1/2     Loss: 5.2702038288116455

Epoch:    1/2     Loss: 4.91266131401062

Epoch:    1/2     Loss: 5.003264904022217

Epoch:    1/2     Loss: 5.262006759643555

Epoch:    1/2     Loss: 5.211731672286987

Epoch:    1/2     Loss: 5.067097902297974

Epoch:    1/2     Loss: 5.068681240081787

Epoch:    1/2     Loss: 5.01192307472229

Epoch:    1/2     Loss: 5.00998854637146

Epoch:    1/2     Loss: 5.068562030792236

Epoch:    1/2     Loss: 5.085802316665649

Epoch:    1/2     Loss: 4.849376201629639

Epoch:    1/2     Loss: 4.907435655593872

Epoch:    1/2     Loss: 5.354885101318359

Epoch:    1/2     Loss: 5.1174232959747314

Epoch:    1/2     Loss: 4.95990252494812

Epoch:    1/2     Loss: 5.255237817764282

Epoch:    1/2     Loss: 5.110088348388672

Epoch:    1/2     Loss: 4.937277555465698

Epoch:    1/2     Loss: 5.0506603717803955

Epoch:    1/2     Loss: 5.414927005767822

Epoch:    1/2     Loss: 5.303488731384277

Epoch:    1/2     Loss: 5.029011964797974

Epoch:    1/

Epoch:    1/2     Loss: 4.930078983306885

Epoch:    1/2     Loss: 5.1813201904296875

Epoch:    1/2     Loss: 4.7460761070251465

Epoch:    1/2     Loss: 4.89687705039978

Epoch:    1/2     Loss: 5.024218559265137

Epoch:    1/2     Loss: 5.033154010772705

Epoch:    1/2     Loss: 5.087931394577026

Epoch:    1/2     Loss: 4.6539788246154785

Epoch:    1/2     Loss: 4.640679121017456

Epoch:    1/2     Loss: 4.746365547180176

Epoch:    1/2     Loss: 4.544464588165283

Epoch:    1/2     Loss: 4.689460515975952

Epoch:    1/2     Loss: 4.915679693222046

Epoch:    1/2     Loss: 4.93556809425354

Epoch:    1/2     Loss: 4.727648735046387

Epoch:    1/2     Loss: 5.024245738983154

Epoch:    1/2     Loss: 5.064399480819702

Epoch:    1/2     Loss: 4.7491455078125

Epoch:    1/2     Loss: 4.860838174819946

Epoch:    1/2     Loss: 4.8170459270477295

Epoch:    1/2     Loss: 4.870080232620239

Epoch:    1/2     Loss: 4.742706298828125

Epoch:    1/2     Loss: 4.913796901702881

Epoch:    1

Epoch:    1/2     Loss: 4.880321025848389

Epoch:    1/2     Loss: 4.778994083404541

Epoch:    1/2     Loss: 4.624528646469116

Epoch:    1/2     Loss: 4.546257019042969

Epoch:    1/2     Loss: 4.5960376262664795

Epoch:    1/2     Loss: 4.40380859375

Epoch:    1/2     Loss: 4.668412685394287

Epoch:    1/2     Loss: 4.748365879058838

Epoch:    1/2     Loss: 4.583006143569946

Epoch:    1/2     Loss: 4.623206615447998

Epoch:    1/2     Loss: 4.781081676483154

Epoch:    1/2     Loss: 4.7173542976379395

Epoch:    1/2     Loss: 4.523968458175659

Epoch:    1/2     Loss: 4.66013503074646

Epoch:    1/2     Loss: 4.579801559448242

Epoch:    1/2     Loss: 4.766413688659668

Epoch:    1/2     Loss: 4.794912815093994

Epoch:    1/2     Loss: 4.879125356674194

Epoch:    1/2     Loss: 4.613538980484009

Epoch:    1/2     Loss: 4.667464733123779

Epoch:    1/2     Loss: 4.788618326187134

Epoch:    1/2     Loss: 4.705225229263306

Epoch:    1/2     Loss: 4.736570835113525

Epoch:    1/2 

Epoch:    1/2     Loss: 4.598155975341797

Epoch:    1/2     Loss: 4.771397829055786

Epoch:    1/2     Loss: 4.53357195854187

Epoch:    1/2     Loss: 4.489098072052002

Epoch:    1/2     Loss: 4.479524850845337

Epoch:    1/2     Loss: 4.676141262054443

Epoch:    1/2     Loss: 4.401248455047607

Epoch:    1/2     Loss: 4.644002914428711

Epoch:    1/2     Loss: 4.6518778800964355

Epoch:    1/2     Loss: 4.172864675521851

Epoch:    1/2     Loss: 4.510862112045288

Epoch:    1/2     Loss: 4.462759733200073

Epoch:    1/2     Loss: 4.7219507694244385

Epoch:    1/2     Loss: 4.418048620223999

Epoch:    1/2     Loss: 4.477855443954468

Epoch:    1/2     Loss: 4.611266374588013

Epoch:    1/2     Loss: 4.625932455062866

Epoch:    1/2     Loss: 4.612622022628784

Epoch:    1/2     Loss: 4.788161754608154

Epoch:    1/2     Loss: 4.638844013214111

Epoch:    1/2     Loss: 4.524443864822388

Epoch:    1/2     Loss: 4.562219619750977

Epoch:    1/2     Loss: 4.624602794647217

Epoch:    

...

Epoch:    2/2     Loss: 4.191867828369141

Epoch:    2/2     Loss: 3.992436170578003

Epoch:    2/2     Loss: 4.325878620147705

Epoch:    2/2     Loss: 4.276460647583008

Epoch:    2/2     Loss: 4.127146244049072

Epoch:    2/2     Loss: 4.155510306358337

Epoch:    2/2     Loss: 4.00453245639801

Epoch:    2/2     Loss: 3.878953456878662

Epoch:    2/2     Loss: 3.9625343084335327

Epoch:    2/2     Loss: 4.192953109741211

Epoch:    2/2     Loss: 4.113512992858887

Epoch:    2/2     Loss: 4.006607532501221

Epoch:    2/2     Loss: 4.24872350692749

Epoch:    2/2     Loss: 4.249274492263794

Epoch:    2/2     Loss: 4.101677060127258

Epoch:    2/2     Loss: 4.06032657623291

Epoch:    2/2     Loss: 4.097837209701538

Epoch:    2/2     Loss: 4.096651077270508

Epoch:    2/2     Loss: 4.29087233543396

Epoch:    2/2     Loss: 4.039341926574707

Epoch:    2/2     Loss: 3.915620803833008

Epoch:    2/2     Loss: 4.243590593338013

Epoch:    2/2     Loss: 4.369452953338623

Epoch:    2/2 

Epoch:    2/2     Loss: 4.042429089546204

Epoch:    2/2     Loss: 4.133110284805298

Epoch:    2/2     Loss: 4.1289976835250854

Epoch:    2/2     Loss: 4.2981462478637695

Epoch:    2/2     Loss: 4.348963499069214

Epoch:    2/2     Loss: 4.032058358192444

Epoch:    2/2     Loss: 3.8924455642700195

Epoch:    2/2     Loss: 4.21593165397644

Epoch:    2/2     Loss: 4.0336912870407104

Epoch:    2/2     Loss: 3.9701627492904663

Epoch:    2/2     Loss: 3.9812276363372803

Epoch:    2/2     Loss: 4.22013783454895

Epoch:    2/2     Loss: 3.924756407737732

Epoch:    2/2     Loss: 4.383921146392822

Epoch:    2/2     Loss: 3.8435158729553223

Epoch:    2/2     Loss: 4.12615442276001

Epoch:    2/2     Loss: 4.192695379257202

Epoch:    2/2     Loss: 4.101222276687622

Epoch:    2/2     Loss: 4.078842639923096

Epoch:    2/2     Loss: 4.2228710651397705

Epoch:    2/2     Loss: 4.222943305969238

Epoch:    2/2     Loss: 4.272746801376343

Epoch:    2/2     Loss: 4.165365099906921

Epoch:

Epoch:    2/2     Loss: 4.3090410232543945

Epoch:    2/2     Loss: 4.3381733894348145

Epoch:    2/2     Loss: 3.9939441680908203

Epoch:    2/2     Loss: 4.4206223487854

Epoch:    2/2     Loss: 4.421051502227783

Epoch:    2/2     Loss: 3.992642402648926

Epoch:    2/2     Loss: 4.109133243560791

Epoch:    2/2     Loss: 4.280481815338135

Epoch:    2/2     Loss: 4.184019565582275

Epoch:    2/2     Loss: 4.244663119316101

Epoch:    2/2     Loss: 4.507559061050415

Epoch:    2/2     Loss: 4.059475779533386

Epoch:    2/2     Loss: 4.3194262981414795

Epoch:    2/2     Loss: 3.8580998182296753

Epoch:    2/2     Loss: 4.175992250442505

Epoch:    2/2     Loss: 4.0990670919418335

Epoch:    2/2     Loss: 4.023872971534729

Epoch:    2/2     Loss: 4.030575156211853

Epoch:    2/2     Loss: 4.199065923690796

Epoch:    2/2     Loss: 4.31153678894043

Epoch:    2/2     Loss: 4.065625190734863

Epoch:    2/2     Loss: 4.191831350326538

Epoch:    2/2     Loss: 4.1732096672058105

Epoch: 

Epoch:    2/2     Loss: 4.1993408203125

Epoch:    2/2     Loss: 4.22524094581604

Epoch:    2/2     Loss: 4.062068939208984

Epoch:    2/2     Loss: 4.204374313354492

Epoch:    2/2     Loss: 4.176886320114136

Epoch:    2/2     Loss: 4.232509136199951

Epoch:    2/2     Loss: 3.8928465843200684

Epoch:    2/2     Loss: 4.010295867919922

Epoch:    2/2     Loss: 4.521107912063599

Epoch:    2/2     Loss: 4.035231232643127

Epoch:    2/2     Loss: 3.999861240386963

Epoch:    2/2     Loss: 4.29458212852478

Epoch:    2/2     Loss: 3.9575194120407104

Epoch:    2/2     Loss: 4.078871011734009

Epoch:    2/2     Loss: 3.839046597480774

Epoch:    2/2     Loss: 3.869702935218811

Epoch:    2/2     Loss: 3.954394817352295

Epoch:    2/2     Loss: 4.255961179733276

Epoch:    2/2     Loss: 4.039650917053223

Epoch:    2/2     Loss: 4.143402338027954

Epoch:    2/2     Loss: 4.29703950881958

Epoch:    2/2     Loss: 3.909224271774292

Epoch:    2/2     Loss: 4.1749348640441895

Epoch:    2/2

/home/jupiter/.virtualenvs/deep-learning-nd/lib/python3.6/site-packages/torch/serialization.py:251: UserWarning: Couldn't retrieve source code for container of type RNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


### Question: How did you decide on your model hyperparameters? 
For example, did you try different sequence_lengths and find that one size made the model converge faster? What about your hidden_dim and n_layers; how did you decide on those?

**Answer:** (Write answer, here)

---
# Checkpoint

After running the above training cell, your model will be saved by name, `trained_rnn`, and if you save your notebook progress, **you can pause here and come back to this code at another time**. You can resume your progress by running the next cell, which will load in our word:id dictionaries _and_ load in your saved model by name!

In [30]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import torch
import helper
import problem_unittests as tests

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
trained_rnn = helper.load_model('./save/trained_rnn')

## Generate TV Script
With the network trained and saved, you'll use it to generate a new, "fake" Seinfeld TV script in this section.

### Generate Text
To generate the text, the network needs to start with a single word and repeat its predictions until it reaches a set length. You'll be using the `generate` function to do this. It takes a word id to start with, `prime_id`, and generates a set length of text, `predict_len`. Also note that it uses topk sampling to introduce some randomness in choosing the most likely next word, given an output set of word scores!

In [31]:
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
import torch.nn.functional as F

def generate(rnn, prime_id, int_to_vocab, token_dict, pad_value, predict_len=100):
    """
    Generate text using the neural network
    :param decoder: The PyTorch Module that holds the trained neural network
    :param prime_id: The word id to start the first prediction
    :param int_to_vocab: Dict of word id keys to word values
    :param token_dict: Dict of puncuation tokens keys to puncuation values
    :param pad_value: The value used to pad a sequence
    :param predict_len: The length of text to generate
    :return: The generated text
    """
    rnn.eval()
    
    # create a sequence (batch_size=1) with the prime_id
    current_seq = np.full((1, sequence_length), pad_value)
    current_seq[-1][-1] = prime_id
    predicted = [int_to_vocab[prime_id]]
    
    for _ in range(predict_len):
        if train_on_gpu:
            current_seq = torch.LongTensor(current_seq).cuda()
        else:
            current_seq = torch.LongTensor(current_seq)
        
        # initialize the hidden state
        hidden = rnn.init_hidden(current_seq.size(0))
        
        # get the output of the rnn
        output, _ = rnn(current_seq, hidden)
        
        # get the next word probabilities
        p = F.softmax(output, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
         
        # use top_k sampling to get the index of the next word
        top_k = 5
        p, top_i = p.topk(top_k)
        top_i = top_i.numpy().squeeze()
        
        # select the likely next word index with some element of randomness
        p = p.numpy().squeeze()
        word_i = np.random.choice(top_i, p=p/p.sum())
        
        # retrieve that word from the dictionary
        word = int_to_vocab[word_i]
        predicted.append(word)     
        
        # the generated word becomes the next "current sequence" and the cycle can continue
        current_seq = np.roll(current_seq, -1, 1)
        current_seq[-1][-1] = word_i
    
    gen_sentences = ' '.join(predicted)
    
    # Replace punctuation tokens
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        gen_sentences = gen_sentences.replace(' ' + token.lower(), key)
    gen_sentences = gen_sentences.replace('\n ', '\n')
    gen_sentences = gen_sentences.replace('( ', '(')
    
    # return all the sentences
    return gen_sentences

### Generate a New Script
It's time to generate the text. Set `gen_length` to the length of TV script you want to generate and set `prime_word` to one of the following to start the prediction:
- "jerry"
- "elaine"
- "george"
- "kramer"

You can set the prime word to _any word_ in our dictionary, but it's best to start with a name for generating a TV script. (You can also start with any other names you find in the original text file!)

In [33]:
# run the cell multiple times to get different results!
gen_length = 400 # modify the length to your preference
prime_word = 'jerry' # name for starting the script

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
pad_word = helper.SPECIAL_WORDS['PADDING']
generated_script = generate(trained_rnn, vocab_to_int[prime_word + ':'], int_to_vocab, token_dict, vocab_to_int[pad_word], gen_length)
print(generated_script)

elaine:, and i don't want to go to me.

george:(looking on the door) : i just know.

elaine: oh, yeah.

george: oh.

jerry:(looking to the phone) : i don't know, but it was the best of this.

jerry: oh, that's not a good time.

kramer: yeah.

elaine: oh, i gotta see you. you know what i want to get the car?

george: i know, i can't believe i have to be it.

newman: well, i'm not gonna get up.

jerry: i think you have to do that.

jerry:(trying to get the table) yeah, i gotta get up.

newman: i think i'm sorry, you know, i was just gonna get a little- five-- i have a little. i think i got it to me. i don't want it.

elaine: i don't know what you want.

kramer: oh, yeah, that's right.

george: i can't get up to him.

george: oh, no.

elaine: yeah, i don't think it's a lot of the other day of a little- and i don't want to be the car to be.

jerry: i got the way, i don't have the same thing.....

jerry: i don't understand it, you don't want to tell you.

jerry: oh, i gotta go.(she turns up

#### Save your favorite scripts

Once you have a script that you like (or find interesting), save it to a text file!

In [ ]:
# save script to a text file
f =  open("generated_script_1.txt","w")
f.write(generated_script)
f.close()

# The TV Script is Not Perfect
It's ok if the TV script doesn't make perfect sense. It should look like alternating lines of dialogue, here is one such example of a few generated lines.

### Example generated script

>jerry: what about me?
>
>jerry: i don't have to wait.
>
>kramer:(to the sales table)
>
>elaine:(to jerry) hey, look at this, i'm a good doctor.
>
>newman:(to elaine) you think i have no idea of this...
>
>elaine: oh, you better take the phone, and he was a little nervous.
>
>kramer:(to the phone) hey, hey, jerry, i don't want to be a little bit.(to kramer and jerry) you can't.
>
>jerry: oh, yeah. i don't even know, i know.
>
>jerry:(to the phone) oh, i know.
>
>kramer:(laughing) you know...(to jerry) you don't know.

You can see that there are multiple characters that say (somewhat) complete sentences, but it doesn't have to be perfect! It takes quite a while to get good results, and often, you'll have to use a smaller vocabulary (and discard uncommon words), or get more data.  The Seinfeld dataset is about 3.4 MB, which is big enough for our purposes; for script generation you'll want more than 1 MB of text, generally. 

# Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_tv_script_generation.ipynb" and save another copy as an HTML file by clicking "File" -> "Download as.."->"html". Include the "helper.py" and "problem_unittests.py" files in your submission. Once you download these files, compress them into one zip file for submission.